In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create synthetic data
np.random.seed(0)
n_samples = 100

# Features
X1 = np.linspace(0, 10, n_samples)  # Increasing feature
X2 = np.linspace(0, 10, n_samples)  # Decreasing feature

# Response with sinusoidal variation
y = 3 * np.sin(X1) + 2 * np.cos(X2) + np.random.normal(0, 0.1, n_samples)

# Create DataFrame
data = pd.DataFrame({'X1': X1, 'X2': X2, 'y': y})
X = data[['X1', 'X2']]
y = data['y']

# Convert to DMatrix
dtrain = xgb.DMatrix(X, label=y)

# Define parameters without monotonic constraints
params_no_constraints = {
    'objective': 'reg:squarederror',
    'booster': 'gbtree',
    'eval_metric': 'rmse'
}

# Define parameters with monotonic constraints
params_constrained = params_no_constraints.copy()
params_constrained['monotone_constraints'] = (1, -1)

# Train model without constraints
model_no_constraints = xgb.train(params_no_constraints, dtrain, num_boost_round=1000)

# Train model with constraints
model_with_constraints = xgb.train(params_constrained, dtrain, num_boost_round=1000)

# Function to plot feature effect
def plot_feature_effect(model, feature_name, title):
    # Create a grid of values for the feature
    x_grid = np.linspace(X[feature_name].min(), X[feature_name].max(), 100)

    # Create a DataFrame for prediction with an index
    data_grid = pd.DataFrame({feature_name: x_grid, 'X2': X['X2'].mean()}, index=range(len(x_grid))) # Added index here

    # Convert DataFrame to DMatrix for predictions
    dgrid = xgb.DMatrix(data_grid)

    # Predict using the model
    y_preds = model.predict(dgrid)

    # Plotting
    plt.figure(figsize=(10, 5))
    plt.plot(x_grid, y_preds, label='Model Prediction')
    plt.xlabel(feature_name)
    plt.ylabel('Prediction')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

# Plot feature effects
plot_feature_effect(model_no_constraints, 'X1', 'No Constraint - Feature X1')
plot_feature_effect(model_no_constraints, 'X2', 'No Constraint - Feature X2')
plot_feature_effect(model_with_constraints, 'X1', 'With Constraint - Feature X1')
plot_feature_effect(model_with_constraints, 'X2', 'With Constraint - Feature X2')
